In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#matplot 한글폰트 설정
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\Windows\Fonts\GULIM.TTC").get_name()
plt.rc("font", family = font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"]= False

## 서울 자치구별 전월세전환율 2018 - 2022년 까지의 %  dataframe 구축
### 월세가를 아파트 전세가로 변환하여 수치를 동일화 시키기 위함
#### 월세에서 전세 전환식 = 월세 보증금 + (월차임 * 12개월) / 지역별 전월세전환율

## 지역별 전월세전환율 Data source 2018~2022
### https://kosis.kr/search/search.do

In [ ]:
# 아파트 매매가격 대비 전세가격 비율 2018 - 2022
apt_rent_rate = pd.read_csv(r"C:/Users/acorn/OneDrive/Desktop/중간프로젝트/부동산/지역별_전월세전환율_2018-2022.csv", encoding='euc-kr')
apt_rent_rate.head()


In [ ]:
#불필요한 columns 확인 - '주택유형별(1)', '지역별(1)', '지역별(2)', '지역별(3)'
apt_rent_rate.columns


In [ ]:
#불필요한 columns 제거 - '주택유형별(1)', '지역별(1)', '지역별(2)', '지역별(3)'

apt_rent_rate.drop(columns=['주택유형별(1)', '지역별(1)',
                   '지역별(2)', '지역별(3)'], inplace=True)
apt_rent_rate


In [ ]:
#시군구 column 자치구별로 변동
apt_rent_rate.rename(columns={'지역별(4)': '자치구별'}, inplace=True)
apt_rent_rate.head()


In [ ]:
#자치구별 column에서 소계 확인
apt_rent_rate['자치구별'].unique()


In [ ]:
#자치구별 column에서 소계가 포함된 행 제거
apt_rent_rate = apt_rent_rate[apt_rent_rate['자치구별'] != '소계']
apt_rent_rate.head()


In [ ]:
#데이터프레임 인덱스 리셋
apt_rent_rate.reset_index(drop=True, inplace=True)
apt_rent_rate.head()


In [ ]:
#'자치구별' column value에 '구' 추가해서 자치구 이름 변경
addname = apt_rent_rate['자치구별'].unique()

#for 구문으로 '자치구별' value에 '구'추가해주기
addname_gu = [name + '구' for name in addname]
apt_rent_rate['자치구별'] = addname_gu

#결과확인
apt_rent_rate.head()


In [ ]:
apt_rent_rate.columns.unique()


In [ ]:
#save point 데이터 저장하기
apt_rent_rate.to_csv(
    r"C:/Users/acorn/OneDrive/Desktop/중간프로젝트/부동산/자치구별_전월세전환율_2018-2022.csv", encoding='utf-8')


In [ ]:
#데이터 불러와서 재확인
df = pd.read_csv(r"C:/Users/acorn/OneDrive/Desktop/중간프로젝트/부동산/자치구별_전월세전환율_2018-2022.csv", encoding='utf-8',index_col=0)
df.head()

In [ ]:
# 2018~2022년도별로 월별 전월세전환율 데이터프레임 생성하여 CSV 파일로 저장
import os

for year in range(2018, 2023):
    year_cols = [col for col in df.columns if col.startswith(str(year) + ".")]
    year_df = df[['자치구별'] + year_cols]

    # 폴더가 없으면 생성
    path = "C:/Users/acorn/OneDrive/Desktop/중간프로젝트/부동산"
    if not os.path.exists(path):
        os.makedirs(path)

    # 파일 저장
    year_df.to_csv(f"{path}/자치구별_전월세전환율_{year}.csv", index=False)


In [ ]:
df.head()

# 아파트 전세, 월세 실거래가 전처리

In [ ]:
#아파트 전세,월세 실거래가 가져오기 및 필요없는 행 제거
rent_df = pd.read_csv(
    r"C:/Users/acorn/OneDrive/Desktop/중간프로젝트/부동산/아파트(전월세)_실거래가_2022_국토교통부.csv", encoding='euc-kr',
    skiprows=15)
rent_df.head()


In [ ]:
rent_df['계약년월'] = rent_df['계약년월'].astype(
    str).apply(lambda x: x[:4] + '.' + x[4:6])
rent_df.info()
rent_df['계약년월']


In [ ]:
rent_df.head()

In [ ]:
#데이터 프레임 카피 만들기
df1 = rent_df.copy()
df1.head()

# 전세, 월세 아파트 전세가로 수치변환
### 아파트 전세 가격일 경우 그대로 전세가로 사용함
### 아파트 월세 가격일 경우
#### 월세를 전세가격으로 변환 후 전세가격
#### 보증금 + (월세 * 12개월) / 자치구별 전월세변환율 = 월세가격 변환

In [ ]:
df1.columns

In [ ]:
df1.isnull().sum()
df1.info()
df1.head()

In [ ]:
df1.head()

In [ ]:
# 보증금(만원)과 월세(만원) column을 숫자형으로 변환
df1['보증금(만원)'] = df1['보증금(만원)'].str.replace(',', '').astype(int)



In [ ]:
df1['월세(만원)'].isnull().sum()


In [ ]:
#월세 결측치 처리
df1['월세(만원)'] = df1['월세(만원)'].str.replace(',','').fillna(0).astype(int)


In [ ]:
df1.info()

In [ ]:
#월세와 보증금 모두 0인 데이터 확인
zero_rent_deposit_count = ((df1['월세(만원)'] == 0) & (df1['보증금(만원)'] == 0)).sum()
print("월세와 보증금 모두 0인 데이터 개수:", zero_rent_deposit_count)


In [ ]:
a = df1['보증금(만원)'].isna().sum()
b = df1['월세(만원)'].isna().sum()
print(a, b)


In [ ]:
#월세와 보증금 모두 0 인 데이터 제거
df1 = df1[(df1['보증금(만원)'] != 0) | (df1['월세(만원)'] != 0)]
df1

In [ ]:
#월세와 보증금 모두 0인 데이터 확인
zero_rent_deposit_count = ((df1['월세(만원)'] == 0) & (df1['보증금(만원)'] == 0)).sum()
print("월세와 보증금 모두 0인 데이터 개수:", zero_rent_deposit_count)


In [ ]:
#리스트 결측치 채워넣기, 확인
df1.index = range(len(df1))
df1


In [ ]:
df1.info()

In [ ]:
# 컬럼의 순서(위치) 바꾸기
cols = ['시군구', '전용면적(㎡)', '계약년월', '전월세구분', '보증금(만원)', '월세(만원)']
df2 = pd.DataFrame(df1, columns = cols)
df2


In [ ]:
#시군구 column 자치구별로 변동
df2.rename(columns = {'시군구':'자치구별'}, inplace = True)

In [ ]:
#자치구별 주소 구로만 가져오기
df2['자치구별'] = df2['자치구별'].str.split().str[1]
df2


In [ ]:
fig = sns.pairplot(data=df2)
fig.fig.set_size_inches (10, 10)
plt.show()


In [ ]:
plt.figure(figsize=(20, 10))
sns.boxplot(x='자치구별', y='보증금(만원)', hue='전월세구분', data=df2)
plt.show()

In [ ]:
#이상치 확인시 df1 불러와서 data detail 보고 df2
df2[df2['자치구별'] == "용산구"].sort_values(by='보증금(만원)', ascending=False)
#df1
df1[df1['보증금(만원)']==490000]


In [ ]:
plt.figure(figsize=(25, 10))
sns.boxplot(x='계약년월', y='보증금(만원)', hue='전월세구분', data=df2)
plt.show()


In [ ]:
#2019년 전환율 불러오기
check_1 = pd.read_csv(
    r"C:/Users/acorn/OneDrive/Desktop/중간프로젝트/부동산/자치구별_전월세전환율_2022.csv")
check_1


In [ ]:
check_1.set_index('자치구별', inplace = True)


In [ ]:
check_1.info()


In [ ]:
df2
df2.info()

In [ ]:
try:
    result = check_1[df2.loc[0]['계약년월']][df2.loc[0]['자치구별']]
except KeyError as e:
    # KeyError 발생 시 처리할 코드 작성
    print(f"KeyError 발생: {e}")
else:
    # 정상적으로 결과가 반환된 경우 처리할 코드 작성
    print(result)


In [ ]:
#확인용
check_1[df2.loc[0]['계약년월']][df2.loc[0]['자치구별']]


In [ ]:
df2.info()

In [ ]:
#자치구별, 계약연도월 전월세전환비율을 df2 row에 추가
for i in range(len(df2)):
    df2.loc[i, '전월세변환율'] = check_1[df2.loc[i]['계약년월']][df2.loc[i]['자치구별']]
    #print(check_1[df2.loc[i]['계약년월']][df2.loc[i]['자치구별']])
    print(i)


In [ ]:
df2


In [ ]:
check_1

In [ ]:
#plt.hist(df2['거래금액(만원)'], bins=30)

sns.distplot(df1['보증금(만원)'], bins = 50)

plt.show()


In [ ]:
sns.distplot(df1['월세(만원)'], bins=30)
plt.show()

In [ ]:
df2

In [ ]:
#월세,전세변환금 = 보증금(만원) + (월세(만원) * 12개월) / 지역별 전월세전환율
# 거래금액(만원) 데이터 변환

df2["거래금액(만원)"] = (df2["보증금(만원)"] + (df2["월세(만원)"]*12)/df2["전월세변환율"]).round(0)

df2


In [ ]:
check_1

In [ ]:
# 전용면적당 거래금액(만원)
df2['1m^2 전용면적당 거래금액(만원)'] = (df2['거래금액(만원)'] / df2['전용면적(㎡)']).astype(int)
df2.head()


In [ ]:
df2.info()

In [ ]:
#계약년월 -> 계약연도, 계약월로 나누기
df2['계약년월'] = df2['계약년월'].str.replace(".", "").astype(int)
df2['계약연도'] = df2['계약년월'] // 100
df2['계약월'] = df2['계약년월'] % 100


In [ ]:
df2.head()

In [ ]:
#중간포인트 저장
df2.to_csv(
    r"C:/seokwonna/프로젝트/save_point2_apt_rent_2022_data.csv", encoding='utf-8')


In [ ]:
df2 = pd.read_csv(r"C:/seokwonna/프로젝트/save_point2_apt_rent_2022_data.csv", index_col = 0)
df2


In [ ]:
df2.columns
df2.info()

In [ ]:
# 컬럼의 순서(위치) 바꾸기
cols = ['자치구별', '계약연도', '계약월', '전월세구분',
        '전용면적(㎡)','거래금액(만원)', '1m^2 전용면적당 거래금액(만원)']
df3 = pd.DataFrame(df2, columns=cols)
df3


In [ ]:
#group_by 사용해서 자치구별 + 계약년월 대비 전용면적당 거래금액(만원) 평균 df으로 구성
df4 = df3.groupby([df3.자치구별, df3.계약월]).mean().round(0)
df4



In [ ]:
unique_values = df3['자치구별'].unique()
print(unique_values)
print(unique_values.size)


In [ ]:
#자치구 월별 매매된 평균 아파트 매매가, 1m^2 전용면적당 평균가
df4.to_csv(r"C:/seokwonna/프로젝트/save_point3_apt_rent_2022_data.csv", encoding='utf-8')


In [ ]:
df5 = df3.groupby([df3.자치구별]).mean().round(0)
df5


In [ ]:
#서울시 수치화된 데이터 평균가 추가
df5.loc['평균'] = df5.mean().round(0)
df5


In [ ]:
#2018년 자치구별 매매된 평균 아파트 전세 및 월세가, 1m^2 전용면적당 평균가
df5.to_csv(
    r"C:/seokwonna/프로젝트/save_point4_apt_rent_2022_data.csv", encoding='utf-8')


In [ ]:
#데이터 불러와서 확인하기
check_1 = pd.read_csv(r"C:/seokwonna/프로젝트/save_point3_apt_rent_2022_data.csv", encoding='utf-8')
check_2 = pd.read_csv(r"C:/seokwonna/프로젝트/save_point4_apt_rent_2022_data.csv", encoding='utf-8')

In [ ]:
check_1

In [ ]:
#계약월 데이터 무시하세요
check_2